# Donwload the data

In [ ]:
!pip install kaggle

In [ ]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
import kaggle
import tensorflow as tf
import pandas as pd

In [ ]:
import io
import os
import re
import shutil
import string
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.layers import TextVectorization

In [ ]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

imdb-dataset-of-50k-movie-reviews.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip /content/imdb-dataset-of-50k-movie-reviews.zip

Archive:  /content/imdb-dataset-of-50k-movie-reviews.zip
replace IMDB Dataset.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


# Preprocessing of Data, Tokenization and Numericalization:

In [ ]:
import pandas as pd

In [ ]:
dataset = pd.read_csv('/content/IMDB Dataset.csv')

In [ ]:
dataset.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
# Create a custom standardization function to strip HTML break tags '<br />'.
def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
  return tf.strings.regex_replace(stripped_html,
                                  '[%s]' % re.escape(string.punctuation), '')


# Vocabulary size and number of words in a sequence.
vocab_size = 60000
sequence_length = 200

# Use the text vectorization layer to normalize, split, and map strings to
# integers. Note that the layer uses the custom standardization defined above.
# Set maximum_sequence length as all samples are not of the same length.
vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=sequence_length)

# Make a text-only dataset (no labels) and call adapt to build the vocabulary.
#text_ds = dataset.map(lambda x, y: x)
vectorize_layer.adapt(dataset.review)

In [ ]:
len(vectorize_layer.get_vocabulary())

60000

In [ ]:
input_data = ["This is an example sentence"]
transformed_data = vectorize_layer(input_data)
print(transformed_data)

tf.Tensor(
[[  11    7   33  462 3813    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0  

In [ ]:
embedding_layer = tf.keras.layers.Embedding(60000, 16)

In [ ]:
embedding_layer(transformed_data).shape

TensorShape([1, 200, 16])

In [ ]:
Labels = []
for i in range(len(dataset.sentiment)):
  if dataset.sentiment[i] == "positive":
    Labels.append(1)
  else:
    Labels.append(0)

In [ ]:
df = pd.DataFrame({'Column1': dataset.sentiment, 'Column2': Labels})

In [ ]:
df.head()

,Column1,Column2
0,positive,1
1,positive,1
2,positive,1
3,negative,0
4,positive,1


In [ ]:
Inputs = vectorize_layer(dataset.review)

In [ ]:
Labels = df.Column2

In [ ]:
# Create a TensorFlow dataset from the vectorized data
Inputs = tf.data.Dataset.from_tensor_slices(Inputs)
Labels = tf.data.Dataset.from_tensor_slices(Labels)

In [ ]:
dataset = tf.data.Dataset.zip((Inputs, Labels))
dataset

<_ZipDataset element_spec=(TensorSpec(shape=(200,), dtype=tf.int64, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>

In [ ]:
#Training and Validation data split :
dataset = dataset.shuffle(len(dataset))
train_size = int(0.8 * len(dataset))
train_dataset = dataset.take(train_size)
val_dataset = dataset.skip(train_size)

In [ ]:
batch_size = 128

# Batch the dataset
train_dataset = train_dataset.batch(batch_size)
val_dataset = val_dataset.batch(batch_size)

In [ ]:
len(train_dataset)

313

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_dataset = train_dataset.cache().prefetch(buffer_size=AUTOTUNE)
val_dataset = val_dataset.cache().prefetch(buffer_size=AUTOTUNE)

# **Training Transformer from scratch**

# Preprocessing Data

In [ ]:
import pandas as pd
dataset = pd.read_csv('/content/IMDB Dataset.csv')

In [ ]:
# Create a custom standardization function to strip HTML break tags '<br />'.
def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
  return tf.strings.regex_replace(stripped_html,
                                  '[%s]' % re.escape(string.punctuation), '')


# Vocabulary size and number of words in a sequence.
vocab_size = 60000
sequence_length = 200

# Use the text vectorization layer to normalize, split, and map strings to
# integers. Note that the layer uses the custom standardization defined above.
# Set maximum_sequence length as all samples are not of the same length.
vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=sequence_length)

# Make a text-only dataset (no labels) and call adapt to build the vocabulary.
#text_ds = dataset.map(lambda x, y: x)
vectorize_layer.adapt(dataset.review)

In [ ]:
Inputs = vectorize_layer(dataset.review)

In [ ]:
Labels = []
for i in range(len(dataset.sentiment)):
  if dataset.sentiment[i] == "positive":
    Labels.append(1)
  else:
    Labels.append(0)

In [ ]:
len(Labels)

50000

In [ ]:
Inputs.shape

TensorShape([50000, 200])

In [ ]:
# Create a TensorFlow dataset from the vectorized data
Inputs = tf.data.Dataset.from_tensor_slices(Inputs)
Labels = tf.data.Dataset.from_tensor_slices(Labels)

In [ ]:
dataset = tf.data.Dataset.zip((Inputs, Labels))
dataset

<_ZipDataset element_spec=(TensorSpec(shape=(200,), dtype=tf.int64, name=None), TensorSpec(shape=(), dtype=tf.int32, name=None))>

In [ ]:
#Training and Validation data split :
dataset = dataset.shuffle(len(dataset))
train_size = int(0.8 * len(dataset))
train_dataset = dataset.take(train_size)
val_dataset = dataset.skip(train_size)

In [ ]:
i = 0
for _, l in train_dataset:
  i= i +1
print(i)

40000


In [ ]:
batch_size = 200
AUTOTUNE = tf.data.AUTOTUNE
# Batch the dataset
train_dataset = train_dataset.batch(batch_size).cache().prefetch(buffer_size=tf.data.AUTOTUNE)
val_dataset = val_dataset.batch(batch_size).cache().prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
i = 0
for _, l in train_dataset:
  i= i +1
print(i)

200


# Positional Embedding

In [ ]:
import numpy as np

In [ ]:
def positional_encoding(length, depth):
  depth = depth/2

  positions = np.arange(length)[:, np.newaxis]     # (seq, 1)
  depths = np.arange(depth)[np.newaxis, :]/depth   # (1, depth)

  angle_rates = 1 / (10000**depths)         # (1, depth)
  angle_rads = positions * angle_rates      # (pos, depth)

  pos_encoding = np.concatenate(
      [np.sin(angle_rads), np.cos(angle_rads)],
      axis=-1)

  return tf.cast(pos_encoding, dtype=tf.float32)

In [ ]:
class PositionalEmbedding(tf.keras.layers.Layer):
  def __init__(self, vocab_size, d_model, length):
    super().__init__()
    self.d_model = d_model
    self.length = length
    self.embedding = tf.keras.layers.Embedding(vocab_size, d_model, mask_zero=True)
    self.pos_encoding = positional_encoding(length = self.length, depth=self.d_model)

  def compute_mask(self, *args, **kwargs):
    return self.embedding.compute_mask(*args, **kwargs)

  def call(self, x):
    length = tf.shape(x)[1]
    x = self.embedding(x)
    # This factor sets the relative scale of the embedding and positonal_encoding.
    x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
    x = x + self.pos_encoding[tf.newaxis, :length, :]
    return x

In [ ]:
for review,_ in train_dataset:
  break

In [ ]:
review

<tf.Tensor: shape=(200, 200), dtype=int64, numpy=
array([[  883,    85,   121, ...,    45,    74,  2091],
       [  539,    42,    11, ...,    33,  3782,   413],
       [ 6506,   693,  1571, ...,     0,     0,     0],
       ...,
       [    2,    60,   280, ...,     0,     0,     0],
       [  104,  1138,  2725, ...,  1040, 16831,    94],
       [   10, 31240,   425, ...,     0,     0,     0]])>

In [ ]:
Embedding = PositionalEmbedding(vocab_size = len(vectorize_layer.get_vocabulary()), d_model = 512, length = 2048)
emb_out = Embedding(review)

In [ ]:
print(review.shape,emb_out.shape)   #((batch,tokens) (batch,tokens,embedding vector))

(200, 200) (200, 200, 512)


# Transformer - Encoder architecture

In [ ]:
import tensorflow as tf

In [ ]:
class Head(tf.keras.Model):
  def __init__(self,head_size,num_heads):
    super().__init__()
    self.key = tf.keras.layers.Dense(head_size)
    self.query = tf.keras.layers.Dense(head_size)
    self.value = tf.keras.layers.Dense(head_size)
    self.sm = tf.keras.layers.Softmax()
    self.dropout = tf.keras.layers.Dropout(0.2)
    self.num_heads = num_heads

  def call(self,x):
    out = []
    B,T,C = x.shape
    for _ in range(self.num_heads):
      k = self.key(x)
      Q = self.query(x)
      V = self.value(x)
      #print(V.shape)
      wei = Q @ tf.transpose(k,perm=[0,2,1]) * C**-0.5
      wei = self.sm(wei)
      #print(wei.shape)
      wei = wei @ V
      out.append(wei)
    out = tf.concat([out[i] for i in range(self.num_heads)], axis=2)
    #print(out.shape)
    return out

In [ ]:
heads = Head(2,3)
matrix_data = [[[1, 2, 3], [4, 5, 6]],[[1, 2, 3], [4, 5, 6]]]
x = tf.constant(matrix_data, dtype=tf.float32)
print(x.shape)
heads(x).shape
#We need to choose num of heads and head_size in order to gives us the same shape of input (B,T,C)

(2, 2, 3)


TensorShape([2, 2, 6])

In [ ]:
class MultiHeadAttention(tf.keras.Model):
  def __init__(self, num_heads, head_size,n_embd,dropout):
    super().__init__()
    self.heads = Head(head_size,num_heads)
    self.proj = tf.keras.layers.Dense(n_embd)
    self.dropout = tf.keras.layers.Dropout(dropout)
    self.head_size = head_size

  def call(self, x):
    x = self.heads(x)
    #x = self.heads(x)  #To add this you need to ensure that the output of the multi head equal the input
    x = self.dropout(self.proj(x))
    return x

In [ ]:
Model = MultiHeadAttention(5,10,10,0.2)
matrix_data = [[[1, 2, 3], [4, 5, 6]],[[1, 2, 3], [4, 5, 6]]]
x = tf.constant(matrix_data, dtype=tf.float32)
print(x.shape)
Model(x).shape
#We need to choose num of heads and head_size in order to gives us the same shape of input (B,T,C)

(2, 2, 3)


TensorShape([2, 2, 10])

In [ ]:
class FeedForward(tf.keras.Model):
  def __init__(self,n_embd,dropout):
    super().__init__()
    self.net = tf.keras.Sequential([
               tf.keras.layers.Dense(4 * n_embd),
               tf.keras.layers.ReLU(),
               tf.keras.layers.Dense(n_embd),
               tf.keras.layers.ReLU(),
               tf.keras.layers.Dense(1,activation = "sigmoid"),
               tf.keras.layers.Dropout(dropout)])
  def call(self, x):
    return self.net(x)

In [ ]:
class Block(tf.keras.Model):
  def __init__(self,num_heads,n_embd,dropout1,dropout2):
    super(Block,self).__init__()
    self.head_size = n_embd // num_heads
    self.sa = MultiHeadAttention(num_heads, self.head_size,n_embd,dropout1)
    self.ffwd = FeedForward(n_embd,dropout2)
    self.ln1 = tf.keras.layers.LayerNormalization()
    self.ln2 = tf.keras.layers.LayerNormalization()
    self.flatten = tf.keras.layers.Flatten()
    self.linear = tf.keras.layers.Dense(1,activation = "sigmoid")
    self.Embedding = PositionalEmbedding(vocab_size = 60000, d_model = 512, length = 2048)

  def call(self,x):
    x = self.Embedding(x)
    x = x + self.sa(self.ln1(x))
    #We need a flatten layer
    x = x + self.ffwd(self.ln2(x))
    x = self.flatten(x)
    #print(x.shape)
    x = self.linear(x)
    return x


In [ ]:
Model = Block(8,512,0.2,0.2)
matrix_data = [[[1, 2, 3,4], [4, 5, 6,7]],[[1, 2, 3,4], [4, 5, 6,7]]]
x = tf.constant(matrix_data, dtype=tf.float32)
print(x.shape)
Model(review).shape
#We need to choose num of heads and head_size in order to gives us the same shape of input (B,T,C)

(2, 2, 4)


TensorShape([200, 1])

In [ ]:
lossclass = tf.keras.losses.BinaryCrossentropy()

In [ ]:
optimizer = tf.keras.optimizers.AdamW(learning_rate = 1e-5)

In [ ]:
num_epochs = 20
for epoch in range(num_epochs):
 i = 0
 v = 0
 j = 0
 totalloss = 0
 Number_of_data = 0
 l = 0
 n = 0
 Number_of_test_data = 0
 for inputs, targets in train_dataset:

    with tf.GradientTape() as tape:
        output = Model(inputs)

        # Compute the loss
        loss = lossclass(targets, output)

    # Compute gradients
    gradients = tape.gradient(loss, Model.trainable_variables)

    # Update the weights
    optimizer.apply_gradients(zip(gradients, Model.trainable_variables))
    totalloss += loss
    for j in range(len(output)):
      if targets[j] == tf.cast(tf.round(output), dtype=tf.int32).numpy()[j]:
        v = v+1
    i += 1
    Number_of_data += len(output)

 for test_inputs ,test_targets in val_dataset:
   test_output = Model(test_inputs)
   for j in range(len(test_output)):
     if test_targets[j] == tf.cast(tf.round(test_output), dtype=tf.int32).numpy()[j]:
        l = l+1
     n +=1
   Number_of_test_data += len(test_output)

 val_accuracy = l/Number_of_test_data
 accuracy = v/(Number_of_data)
 print(f"Epoch {epoch+1}: Loss = {totalloss / i}, number of batch in the data : {i} ,Number of data: {Number_of_data}, accuracy: {accuracy},val_accuracy: {val_accuracy}")

Epoch 1: Loss = 0.1564716398715973, number of batch in the data : 200 ,Number of data: 40000, accuracy: 0.948925,val_accuracy: 0.9371
Epoch 2: Loss = 0.13516134023666382, number of batch in the data : 200 ,Number of data: 40000, accuracy: 0.95775,val_accuracy: 0.9454
Epoch 3: Loss = 0.11602993309497833, number of batch in the data : 200 ,Number of data: 40000, accuracy: 0.964975,val_accuracy: 0.9507
Epoch 4: Loss = 0.09918622672557831, number of batch in the data : 200 ,Number of data: 40000, accuracy: 0.9716,val_accuracy: 0.9556
Epoch 5: Loss = 0.08565936237573624, number of batch in the data : 200 ,Number of data: 40000, accuracy: 0.97645,val_accuracy: 0.9602
Epoch 6: Loss = 0.07874328643083572, number of batch in the data : 200 ,Number of data: 40000, accuracy: 0.97775,val_accuracy: 0.9303
Epoch 7: Loss = 0.08450919389724731, number of batch in the data : 200 ,Number of data: 40000, accuracy: 0.97195,val_accuracy: 0.9647
Epoch 8: Loss = 0.06490463018417358, number of batch in the da

In [ ]:
text = "the only beautiful thing in this movie is the CGI"

In [ ]:
tex = vectorize_layer(text)

In [ ]:
tex = tf.expand_dims(tex, axis=0)

In [ ]:
if tf.round(Model(tex)) == 1:
  print("Positive review")
else:
  print("Negative review")

Negative review
